In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, median_absolute_error, roc_auc_score
import time
from sklearn.preprocessing import LabelEncoder
# We may want to scale the features, especially for Naive Bayes classification
from sklearn.preprocessing import StandardScaler

In [2]:
class NaiveBayesClassifier:
    def __init__(self, dataset, train_size=0.2):
        """
        Initialize with dataset and test size for splitting data.
        """
        self.dataset = dataset
        self.train_size = train_size

        # Initialize the classifier
        self.model = GaussianNB()

        # Initialize evaluation metrics
        self.accuracy = None
        self.sensitivity = None
        self.specificity = None
        self.ppv = None
        self.rmse = None
        self.mae = None
        self.mdae = None
        self.training_time = None
        self.testing_time = None
        self.r2_score = None

    def splitData(self):
        """
        Split data into training and testing sets based on the test size.
        """
        X = self.dataset.iloc[:, :-1]  # Features
        y = self.dataset.iloc[:, -1]   # Labels

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, train_size=self.train_size, random_state=42)

    def fit(self):
        """
        Train the Naive Bayes classifier on the training data.
        """
        start_time = time.time()
        self.model.fit(self.X_train, self.y_train)
        self.training_time = time.time() - start_time

    def predict(self):
        """
        Predict the labels for the test data.
        """
        start_time = time.time()
        self.y_pred = self.model.predict(self.X_test)
        self.testing_time = time.time() - start_time

    def evaluate_model(self):
        """
        Evaluate the model using various metrics.
        """
        # Accuracy
        self.accuracy = accuracy_score(self.y_test, self.y_pred)

        # Confusion Matrix
        cm = confusion_matrix(self.y_test, self.y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Sensitivity (Recall)
        self.sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0

        # Specificity
        self.specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

        # Positive Predictive Value (PPV) or Precision
        self.ppv = tp / (tp + fp) if (tp + fp) != 0 else 0

        # R-squared (Coefficient of determination)
        self.r2_score = self.model.score(self.X_test, self.y_test)

        # RMSE (Root Mean Squared Error)
        self.rmse = np.sqrt(mean_squared_error(self.y_test, self.y_pred))

        # MAE (Mean Absolute Error)
        self.mae = mean_absolute_error(self.y_test, self.y_pred)

        # Median Absolute Error (MdAE)
        self.mdae = median_absolute_error(self.y_test, self.y_pred)

    def start(self):
        """
        Execute the full pipeline: Split data, fit the model, and predict.
        """
        self.splitData()
        self.fit()
        self.predict()
        self.evaluate_model()

        # Print results
        # self.print_results()

    def print_results(self):
        """
        Display the evaluation metrics.
        """
        print(f"Training Time: {self.training_time:.4f} seconds")
        print(f"Testing Time: {self.testing_time:.4f} seconds")
        print(f"Accuracy: {self.accuracy:.4f}")
        print(f"Sensitivity (Recall): {self.sensitivity:.4f}")
        print(f"Specificity: {self.specificity:.4f}")
        print(f"Positive Predictive Value (PPV): {self.ppv:.4f}")
        print(f"R-squared: {self.r2_score:.4f}")
        print(f"RMSE (Root Mean Squared Error): {self.rmse:.4f}")
        print(f"MAE (Mean Absolute Error): {self.mae:.4f}")
        print(f"Median Absolute Error (MdAE): {self.mdae:.4f}")


In [3]:
import yfinance as yf

def getData():
  # Download stock data (e.g., Apple stock) for the last 6 months
  data = yf.download('AAPL', start='2023-08-01', end='2025-01-31')

  # Preview the dataset
  # print(data.head())

  # Add a column 'PriceMovement' to label the movement (up or down)
  data['PriceMovement'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)

  # Drop rows with missing values (NaN values due to shifting)
  data.dropna(inplace=True)

  # Selecting features (you can choose more features as necessary)
  features = ['Open', 'High', 'Low', 'Close', 'Volume']

  # Prepare features (X) and labels (y)
  X = data[features]
  y = data['PriceMovement']


  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  # Convert the scaled features back to a DataFrame for easier merging
  X_scaled_df = pd.DataFrame(X_scaled, columns=features, index=data.index)

  # Merge the scaled features (X_scaled_df) with the target labels (y)
  final_dataset = X_scaled_df.copy()
  final_dataset['PriceMovement'] = y

  return final_dataset

In [8]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, median_absolute_error

class ModelAnalyzer:
    def __init__(self, dataset, train_size):
        """
        Initialize ModelAnalyzer with dataset and list of splits for training/testing.
        """
        self.dataset = dataset
        self.train_size = train_size  # List of different test/train splits (e.g., [0.2, 0.3, 0.4])
        self.results = []
        #self.results.append([''] + self.train_size)
        self.results.append(['Testing Time'] )
        self.results.append(['Training Time'] )
        self.results.append(['Accuracy'] )
        self.results.append(['Sensitivity (Recall)'] )
        self.results.append(['Specificity'] )
        self.results.append(['Positive Predictive Value (PPV)'] )
        self.results.append(['RMSE'] )
        self.results.append(['MAE'] )
        self.results.append(['R2 Score'] )


    def start(self):
        """
        Iterates over different splits, runs the NaiveBayesClassifier, and stores the results.
        """
        for size in self.train_size:
            model = NaiveBayesClassifier(self.dataset, train_size= (0.7 * float(size)))
            model.start()

            # Append the results to the respective lists
            self.results[0].append(model.testing_time)
            self.results[1].append(model.training_time)
            self.results[2].append(model.accuracy)
            self.results[3].append(model.sensitivity)
            self.results[4].append(model.specificity)
            self.results[5].append(model.ppv)
            self.results[6].append(model.rmse)
            self.results[7].append(model.mae)
            self.results[8].append(model.r2_score)


    def get_table(self):
        # Return the list results
        splits = ["train % -->"] + self.train_size

        # Create a pandas DataFrame
        df = pd.DataFrame(self.results, columns=splits)

        # Use pandas styling for a nice table display
        styled_df = df.style.set_table_styles(
            [{'selector': 'thead th',
              'props': [('background-color', '#4CAF50'),
                        ('color', 'white'),
                        ('font-weight', 'bold')]},
            {'selector': 'tbody tr:nth-child(odd)',
              'props': [('background-color', '#f2f2f2')]},
            {'selector': 'tbody tr:nth-child(even)',
              'props': [('background-color', '#ffffff')]},
             {'selector': 'td', 'props': [('text-align', 'left')]}]
        )

        # Display the table
        return(styled_df.hide(axis="index"))


In [9]:
dataset = getData()

train_size=["0.2", "0.3", "0.4", "0.5", "0.6", "0.7"]

# Initialize ModelAnalyzer with the dataset and list of splits
model_analyzer = ModelAnalyzer(dataset, train_size)

# Start the analysis
model_analyzer.start()

# Get the results table
styled_table = model_analyzer.get_table()

# To display the table in Jupyter (or similar environments):
styled_table


[*********************100%***********************]  1 of 1 completed


train % -->,0.2,0.3,0.4,0.5,0.6,0.7
Testing Time,0.001095,0.001050,0.001459,0.001070,0.001402,0.001515
Training Time,0.001953,0.001726,0.001740,0.001926,0.001653,0.002844
Accuracy,0.507692,0.469799,0.474265,0.455285,0.442922,0.590674
Sensitivity (Recall),0.641304,0.374269,0.609272,0.453901,0.433071,0.913043
Specificity,0.333333,0.598425,0.305785,0.457143,0.456522,0.115385
Positive Predictive Value (PPV),0.556604,0.556522,0.522727,0.528926,0.523810,0.603448
RMSE,0.701646,0.728149,0.725076,0.738048,0.746376,0.639786
MAE,0.492308,0.530201,0.525735,0.544715,0.557078,0.409326
R2 Score,0.507692,0.469799,0.474265,0.455285,0.442922,0.590674
